In [6]:
import pandas as pd
customers = pd.read_csv(r'..\data\processed\customers_cleaned.csv')
transactions = pd.read_csv(r'..\data\raw\transactions.csv')   

In [7]:
features = transactions.groupby('customer_id').agg({'transaction_id': 'count'}).reset_index()
features.columns = ['customer_id', 'total_transactions']

In [8]:
features.to_csv(r'..\data\processed\features.csv', index=False)   